# Understanding the ARE Framework

This notebook walks through the core concepts of Meta's Agents Research Environments (ARE) framework.

## What You'll Learn
1. How to load and inspect scenarios
2. Understanding apps and tools
3. Examining the event flow and DAG structure
4. Running scenarios in oracle mode (ground truth)
5. Analyzing agent execution traces

In [ ]:
# Install ARE if not already installed
# !pip install meta-agents-research-environments

In [ ]:
import sys
sys.path.insert(0, '../meta-agents-research-environments')

from are.simulation.environment import Environment, EnvironmentConfig
from are.simulation.scenarios.registration import get_scenario_by_id
from are.simulation.types import EventType
import json

## 1. Loading and Inspecting a Scenario

Let's start by loading a simple scenario to understand its structure.

In [ ]:
# Load a simple scenario
scenario = get_scenario_by_id("scenario_find_image_file")

print(f"Scenario ID: {scenario.scenario_id}")
print(f"Status: {scenario.status}")
print(f"Benchmark Ready: {scenario.is_benchmark_ready}")
print(f"Duration: {scenario.duration} seconds")
print(f"Time Increment: {scenario.time_increment_in_seconds} second(s)")

In [ ]:
# Initialize the scenario (this sets up apps and events)
scenario.initialize()

print(f"\nInitialized: {scenario._initialized}")
print(f"Number of Apps: {len(scenario.apps) if scenario.apps else 0}")
print(f"Number of Events: {len(scenario.events)}")

## 2. Understanding Apps and Tools

Apps are interactive applications that provide APIs for agents to interact with.

In [ ]:
# List all apps in the scenario
print("Available Apps:")
print("="*50)
for app in scenario.apps or []:
    print(f"\n{app.name} ({app.__class__.__name__})")
    print(f"  Description: {app.__doc__[:100] if app.__doc__ else 'No description'}...")

In [ ]:
# Get all tools (functions) available to the agent
tools = scenario.get_tools()

print(f"\nTotal Tools Available: {len(tools)}")
print("\nTool Details:")
print("="*80)

for i, tool in enumerate(tools[:5], 1):  # Show first 5 tools
    print(f"\n{i}. {tool.name}")
    print(f"   Description: {tool.function_description[:100]}...")
    print(f"   Parameters:")
    schema = tool.parameters_schema
    if 'properties' in schema:
        for param_name, param_info in list(schema['properties'].items())[:3]:
            print(f"      - {param_name}: {param_info.get('type', 'unknown')}")

## 3. Examining Events and Event Flow

Events form a DAG (Directed Acyclic Graph) showing what should happen and when.

In [ ]:
# Analyze event structure
print(f"Total Events: {len(scenario.events)}\n")

# Group events by type
from collections import defaultdict
events_by_type = defaultdict(list)

for event in scenario.events:
    events_by_type[type(event).__name__].append(event)

print("Events by Type:")
for event_type, events in events_by_type.items():
    print(f"  {event_type}: {len(events)}")

In [ ]:
# Examine event dependencies (DAG structure)
print("\nEvent Flow (DAG Structure):")
print("="*80)

for event in scenario.events[:10]:  # First 10 events
    print(f"\n{event.event_id}")
    print(f"  Type: {type(event).__name__}")
    print(f"  Relative Time: {event.event_relative_time}s")
    print(f"  Dependencies: {[e.event_id for e in event.dependencies]}")
    print(f"  Successors: {[e.event_id for e in event.successors]}")
    
    # For oracle events, show what action should be taken
    if hasattr(event, 'action_desc') and event.action_desc:
        print(f"  Action: {event.action_desc.app}.{event.action_desc.function}")
        if event.action_desc.args:
            print(f"  Args: {[(arg['name'], arg['value'][:50] if isinstance(arg['value'], str) else arg['value']) for arg in event.action_desc.args[:2]]}")

## 4. Running a Scenario in Oracle Mode

Oracle mode runs the scenario with pre-defined agent actions (ground truth).

In [ ]:
# Create environment in oracle mode
env = Environment(
    config=EnvironmentConfig(
        start_time=0,
        duration=scenario.duration,
        time_increment_in_seconds=scenario.time_increment_in_seconds,
        oracle_mode=True,  # Run with ground truth actions
        queue_based_loop=True,  # Jump to events (faster)
        exit_when_no_events=True,  # Exit when done
        verbose=False  # Reduce output
    )
)

# Run the scenario
print("Running scenario in oracle mode...")
env.run(scenario, wait_for_end=True, schedule_events=True)
print(f"\nEnvironment final state: {env.state}")

## 5. Analyzing Execution Traces

After execution, we can examine what happened at each step.

In [ ]:
# Get event log (completed events)
completed_events = env.event_log.list_view()

print(f"Total Completed Events: {len(completed_events)}\n")

# Analyze by event type
agent_events = [e for e in completed_events if e.event_type == EventType.AGENT]
env_events = [e for e in completed_events if e.event_type == EventType.ENV]
user_events = [e for e in completed_events if e.event_type == EventType.USER]

print(f"Agent Actions: {len(agent_events)}")
print(f"Environment Events: {len(env_events)}")
print(f"User Events: {len(user_events)}")

In [ ]:
# Show detailed trace of agent actions
print("\nAgent Action Trace:")
print("="*80)

for i, event in enumerate(agent_events[:10], 1):
    print(f"\n{i}. Time: {event.event_time:.1f}s")
    
    if event.action:
        print(f"   Tool: {event.action.class_name}.{event.action.function_name}")
        print(f"   Args: {json.dumps(event.action.args, indent=6)[:200]}...")
        print(f"   Result: {str(event.metadata.return_value)[:100]}...")
        
        if event.metadata.exception:
            print(f"   Exception: {event.metadata.exception}")

In [ ]:
# Visualize tool usage frequency
from collections import Counter

tool_usage = Counter()
for event in agent_events:
    if event.action:
        tool_name = f"{event.action.class_name}.{event.action.function_name}"
        tool_usage[tool_name] += 1

print("\nTool Usage Frequency:")
print("="*80)
for tool, count in tool_usage.most_common(10):
    print(f"{tool}: {count} times")

In [ ]:
# Get environment state at end
final_state = env.get_state()

print("\nFinal Environment State:")
print("="*80)
print(f"Current Time: {final_state['current_time']}s")
print(f"Total Events Processed: {len(final_state['event_log']['past_events'])}")
print(f"Remaining Events: {len(final_state['event_queue']['events'])}")

print("\nApp States:")
for app_state in final_state['apps'][:3]:  # Show first 3 apps
    print(f"\n{app_state['app_name']}:")
    # Show some state details (excluding large data)
    for key, value in list(app_state.items())[:5]:
        if key != 'app_name':
            print(f"  {key}: {str(value)[:100]}...")

## Key Takeaways

1. **Scenarios** encapsulate tasks, apps, events, and validation logic
2. **Apps** provide tools (APIs) for agents to interact with
3. **Events** form a DAG showing what should happen and when
4. **Oracle mode** shows ground truth behavior for validation
5. **Event logs** capture everything that happened during execution

## Next Steps

- Explore `02_running_custom_agent.ipynb` to learn how to implement your own agent
- Check `03_agent_with_memory.ipynb` for advanced memory-augmented agents
- Review the CUSTOM_AGENT_GUIDE.md for comprehensive documentation